In [ ]:
!nvidia-smi

## Download COCO 

In [ ]:
#!rm -rf sample_data

In [ ]:
#!mkdir COCO

In [ ]:
#!wget -c --output-document="COCO/train2017.zip" http://images.cocodataset.org/zips/train2017.zip

In [ ]:
#!unzip -q COCO/train2017.zip -d COCO

In [ ]:
#!rm -rf COCO/train2017.zip

In [ ]:
#!wget -c --output-document="COCO/val2017.zip" http://images.cocodataset.org/zips/val2017.zip

In [ ]:
#!unzip -q COCO/val2017.zip -d COCO

In [ ]:
#!rm -rf COCO/val2017.zip

In [ ]:
#!wget -c --output-document="COCO/annotations_trainval2017.zip" http://images.cocodataset.org/annotations/annotations_trainval2017.zip

In [ ]:
#!unzip -q COCO/annotations_trainval2017.zip -d COCO

In [ ]:
#!rm -rf COCO/annotations_trainval2017.zip

CODING CONVENTIONS

- All names use underscore casing
- L_* for list variable names
- D_* for dictionary variable names
- T_* for tensor variable names
- M_* for Pytorch models
- DL_* for Data Loader
- DSET_* for Dataset
- Class names are in upper case
- Function names start with a capital letter

In [ ]:
from torchvision import transforms
from PIL import Image
import os
from os import path
import torch
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from Dataset.CocoROIDataset import COCO_ROI_DATASET
from utils import Denormalize_tensor, Show_figure_with_bbox
from torch.utils.data import DataLoader

## Dataset Class

In [ ]:
exp_name = "RN50-SGD-CyclicLR-DA"
epochs = 200
batch_size = 10
num_batch_train = 400
num_batch_val = 100
lr = 0.006
momentum = 0.6
parameters_path = path.join("parameters", exp_name)
plots_path = path.join("plots", exp_name)

In [ ]:
#DSET_coco_training = COCO_ROI_DATASET(root_dirpath='F:/Datasets/COCO', split='train', batch_size=batch_size, num_batch=num_batch_train)
DSET_coco_validation = COCO_ROI_DATASET(root_dirpath='F:/Datasets/COCO', split='val', batch_size=batch_size, num_batch=num_batch_val)

In [ ]:
(T_img, T_bbox), class_idx = DSET_coco_training[0]
denormalize_func = Denormalize_tensor(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
T_normal_img = denormalize_func(T_img)
Show_figure_with_bbox(transforms.ToPILImage()(T_normal_img), (T_img.shape[1:3]), T_bbox.numpy())
DSET_coco_validation.Get_class_idx_description(class_idx)

In [ ]:
(T_img, T_bbox), class_idx = DSET_coco_validation[0]
denormalize_func = Denormalize_tensor(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
T_normal_img = denormalize_func(T_img)
Show_figure_with_bbox(transforms.ToPILImage()(T_normal_img), (T_img.shape[1:3]), T_bbox.numpy())
DSET_coco_validation.Get_class_idx_description(class_idx)

## Model

In [ ]:
from torchvision.models import resnet50, resnet101
from torchvision.ops import RoIPool
from torchvision.ops import RoIAlign

class M_ROI_CLASSIFIER(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(M_ROI_CLASSIFIER, self).__init__()

        M_backbone = resnet50(pretrained=True, replace_stride_with_dilation=[True, True, True])

        M_conv_ = torch.nn.Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
        M_batchn_ = torch.nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        M_relu_ = torch.nn.ReLU(inplace=True)
        M_custom_layer = torch.nn.Sequential(
            M_conv_,
            M_batchn_,
            M_relu_
        )
        self.M_custom_backbone = torch.nn.Sequential(
            M_backbone.conv1,
            M_backbone.bn1,
            M_backbone.relu,
            M_backbone.maxpool,
            M_backbone.layer1,
            M_backbone.layer2,
            M_backbone.layer3,
            M_backbone.layer4,
            M_custom_layer
        )

        self.M_roi_align = RoIAlign(output_size=(7, 7), spatial_scale=1, sampling_ratio=-1)
        self.M_flatten = torch.nn.Flatten()
        self.M_classifier = torch.nn.Linear(in_features=25088, out_features=80, bias=True)

    #def forward(self, T_image, T_roi_bbox):
    def forward(self, inputs):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        T_image, T_roi_bbox = inputs;
        T_extracted_features = self.M_custom_backbone(T_image)
        batch_size = T_roi_bbox.shape[0]
        T_roi_bbox = T_roi_bbox*T_extracted_features.shape[2];
        T_batch_indices = torch.Tensor(list(range(batch_size))).to("cuda")
        T_roi_bbox = torch.cat((T_batch_indices.view(batch_size, 1), T_roi_bbox), axis=1)
        T_roi_features = self.M_roi_align(T_extracted_features, T_roi_bbox)
        T_roi_features_vector = self.M_flatten(T_roi_features)
        return self.M_classifier(T_roi_features_vector)
        
        """
        batch_size = T_roi_bbox.shape[0]
        T_roi_bbox = T_roi_bbox*240;
        T_batch_indices = torch.Tensor(list(range(batch_size))).to("cuda")
        T_roi_bbox = torch.cat((T_batch_indices.view(batch_size, 1), T_roi_bbox), axis=1)
        T_roi_features = self.M_roi_align(T_image, T_roi_bbox)
        return T_roi_features
        """

In [ ]:
M_roi_classifier = M_ROI_CLASSIFIER().to('cuda')

In [ ]:
DL_coco_training = DataLoader(DSET_coco_training, batch_size=batch_size, shuffle=False)
DL_coco_validation = DataLoader(DSET_coco_validation, batch_size=batch_size, shuffle=False)

## Trainer

In [ ]:
from torchtrainer.trainer import Trainer, Mode
from torchtrainer.callbacks.calculateaccuracycallback import CalculateTopNAccuracyCallback
from torchtrainer.callbacks.calculatelosscallback import CalculateLossCallback
from torchtrainer.callbacks.plotcallback import PlotCallback
from torchtrainer.callbacks.saveparameterscallback import SaveParametersCallback
from torchtrainer.callbacks.settqdmbardescription import SetTQDMBarDescription
from torchtrainer.callbacks.lrbatchschedulercallback import LRBatchSchedulerCallBack

In [ ]:
def prepare_batch_fn(batch, gt):
    gt = gt.long()
    return batch, gt

In [ ]:
optimizer = torch.optim.SGD(M_roi_classifier.parameters(), lr=0.005, momentum=0.2)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer,
                                              step_size_up=50,
                                              base_lr=0.0001, max_lr=0.005,
                                              cycle_momentum=True,
                                              base_momentum=0.1, max_momentum=0.2,
                                             )

trainer = Trainer(
  device = 'cuda',
  modes = [Mode.TRAIN, Mode.EVALUATE],
  model = M_roi_classifier,
  data_loaders = {Mode.TRAIN : DL_coco_training, Mode.EVALUATE : DL_coco_validation},
  epochs = 200,
  starting_epoch = 0,
  optimizer = optimizer,
  criterion = criterion,
  prepare_batch_fn = prepare_batch_fn,
  callbacks = [
    LRBatchSchedulerCallBack(scheduler),
    CalculateLossCallback(key='Loss'),
    CalculateTopNAccuracyCallback(keys=('Top-1 accuracy',), topk=(1,)),
    PlotCallback(plots_path, labels_map={Mode.TRAIN:"Train", Mode.EVALUATE:"Val"}, columns=['Loss', 'Top-1 accuracy']),
    SetTQDMBarDescription(keys=['Loss', 'Top-1 accuracy']),
    SaveParametersCallback(parameters_path),
  ]
)

In [ ]:
trainer.start()

In [ ]:
y=[]
for i in range(400):
    scheduler.step()
    y.append(scheduler.get_lr()[0])
    
plt.plot(range(400),y)
plt.show()

In [ ]:
from torch_lr_finder import LRFinder

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.SGD(M_roi_classifier.parameters(), lr=1e-7, momentum=0.0)
lr_finder = LRFinder(M_roi_classifier, optimizer, criterion, device="cuda")
lr_finder.range_test(DL_coco_training, end_lr=1, num_iter=120)
lr_finder.plot() # to inspect the loss-learning rate graph
lr_finder.reset() # to reset the model and optimizer to their initial state

In [ ]:
optimizer = torch.optim.SGD(M_roi_classifier.parameters(), lr=1e-7, momentum=0.2)
lr_finder = LRFinder(M_roi_classifier, optimizer, criterion, device="cuda")
lr_finder.range_test(DL_coco_training, end_lr=1, num_iter=120)
lr_finder.plot() # to inspect the loss-learning rate graph
lr_finder.reset() # to reset the model and optimizer to their initial state

In [ ]:
optimizer = torch.optim.SGD(M_roi_classifier.parameters(), lr=1e-7, momentum=0.8)
lr_finder = LRFinder(M_roi_classifier, optimizer, criterion, device="cuda")
lr_finder.range_test(DL_coco_training, end_lr=1, num_iter=120)
lr_finder.plot() # to inspect the loss-learning rate graph
lr_finder.reset() # to reset the model and optimizer to their initial state

In [ ]:
optimizer = torch.optim.SGD(M_roi_classifier.parameters(), lr=1e-7, momentum=0.0)
lr_finder = LRFinder(M_roi_classifier, optimizer, criterion, device="cuda")
lr_finder.range_test(DL_coco_training, val_loader=DL_coco_validation, end_lr=1, num_iter=8, step_mode="linear")
lr_finder.plot(log_lr=False)
lr_finder.reset()

In [ ]:
optimizer = optim.Adam(M_roi_classifier.parameters(), lr=0.1, weight_decay=1e-2)
lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
lr_finder.range_test(DL_coco_training, val_loader=DL_coco_validation, end_lr=1, num_iter=100, step_mode="linear")
lr_finder.plot(log_lr=False)
lr_finder.reset()